Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df);
first(df2, 5)

5×4 DataFrame
│ Row │ height   │ weight   │ age      │ male   │
│     │ Float64⍰ │ Float64⍰ │ Float64⍰ │ Int64⍰ │
├─────┼──────────┼──────────┼──────────┼────────┤
│ 1   │ 151.765  │ 47.8256  │ 63.0     │ 1      │
│ 2   │ 139.7    │ 36.4858  │ 63.0     │ 0      │
│ 3   │ 136.525  │ 31.8648  │ 65.0     │ 0      │
│ 4   │ 156.845  │ 53.0419  │ 41.0     │ 1      │
│ 5   │ 145.415  │ 41.2769  │ 51.0     │ 0      │

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchains);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height],
  "weight" => df2[:weight]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME)
# Describe the draws
describe(chn)


File /Users/rob/.julia/dev/StanModels/scripts/04/tmp/weights.stan will be updated.



2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean     │ std       │ naive_se    │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64   │ Float64     │ Float64    │ Any    │
├─────┼────────────┼──────────┼───────────┼─────────────┼────────────┼────────┤
│ 1   │ alpha      │ 113.907  │ 1.90947   │ 0.0301914   │ 0.0499363  │ 4000.0 │
│ 2   │ beta       │ 0.904476 │ 0.0422603 │ 0.000668194 │ 0.00112132 │ 4000.0 │
│ 3   │ sigma      │ 5.10319  │ 0.194797  │ 0.00308     │ 0.00561929 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%     │ 25.0%    │ 50.0%    │ 75.0%   │ 97.5%    │
│     │ Symbol     │ Float64  │ Float64  │ Float64  │ Float64 │ Float64  │
├─────┼────────────┼──────────┼──────────┼──────────┼─────────┼──────────┤
│ 1   │ alpha      │ 110.197  │ 112.608  │ 113.872  │ 115.196 │ 117.636  │
│ 2   │ beta       │ 0.821454 │ 0.875118 │ 0.905213 │ 0.93311 │ 0.985589 │
│ 3   │ sigma      │ 4.72913  │ 4.96807  │ 5.09976  │ 5.23559 │ 5.49519  │


Save the chains in a JLD file

In [7]:
serialize("m4.3s.jls", chn)

chn2 = deserialize("m4.3s.jls")

describe(chn2)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean     │ std       │ naive_se    │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64   │ Float64     │ Float64    │ Any    │
├─────┼────────────┼──────────┼───────────┼─────────────┼────────────┼────────┤
│ 1   │ alpha      │ 113.907  │ 1.90947   │ 0.0301914   │ 0.0499363  │ 4000.0 │
│ 2   │ beta       │ 0.904476 │ 0.0422603 │ 0.000668194 │ 0.00112132 │ 4000.0 │
│ 3   │ sigma      │ 5.10319  │ 0.194797  │ 0.00308     │ 0.00561929 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%     │ 25.0%    │ 50.0%    │ 75.0%   │ 97.5%    │
│     │ Symbol     │ Float64  │ Float64  │ Float64  │ Float64 │ Float64  │
├─────┼────────────┼──────────┼──────────┼──────────┼─────────┼──────────┤
│ 1   │ alpha      │ 110.197  │ 112.608  │ 113.872  │ 115.196 │ 117.636  │
│ 2   │ beta       │ 0.821454 │ 0.875118 │ 0.905213 │ 0.93311 │ 0.985589 │
│ 3   │ sigma      │ 4.72913  │ 4.96807  │ 5.09976  │ 5.23559 │ 5.49519  │


Should be identical to earlier result

In [8]:
describe(chn2)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean     │ std       │ naive_se    │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64   │ Float64     │ Float64    │ Any    │
├─────┼────────────┼──────────┼───────────┼─────────────┼────────────┼────────┤
│ 1   │ alpha      │ 113.907  │ 1.90947   │ 0.0301914   │ 0.0499363  │ 4000.0 │
│ 2   │ beta       │ 0.904476 │ 0.0422603 │ 0.000668194 │ 0.00112132 │ 4000.0 │
│ 3   │ sigma      │ 5.10319  │ 0.194797  │ 0.00308     │ 0.00561929 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%     │ 25.0%    │ 50.0%    │ 75.0%   │ 97.5%    │
│     │ Symbol     │ Float64  │ Float64  │ Float64  │ Float64 │ Float64  │
├─────┼────────────┼──────────┼──────────┼──────────┼─────────┼──────────┤
│ 1   │ alpha      │ 110.197  │ 112.608  │ 113.872  │ 115.196 │ 117.636  │
│ 2   │ beta       │ 0.821454 │ 0.875118 │ 0.905213 │ 0.93311 │ 0.985589 │
│ 3   │ sigma      │ 4.72913  │ 4.96807  │ 5.09976  │ 5.23559 │ 5.49519  │


End of `m4.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*